In [167]:
import pandas as pd
import os,sys,re
import numpy as np
from collections import OrderedDict
os.getcwd()

'/Users/itaybd/covid19/COVID-19/new_covid/COVID-19'

In [158]:
df = pd.read_csv("csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
df.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '10/8/20', '10/9/20', '10/10/20', '10/11/20', '10/12/20', '10/13/20',
       '10/14/20', '10/15/20', '10/16/20', '10/17/20'],
      dtype='object', length=274)

In [159]:
df["key"] = df.apply(lambda r: str(r['Province/State'])+"_"+str(r['Country/Region']),axis=1)
df = df.set_index("key")
df_mod = df[df.columns[4:]]

In [160]:
df_mod_t = df_mod.T
(df_mod_t[df_mod_t.columns[:4]]).head(),(df_mod_t[df_mod_t.columns[:4]]).tail()


(key      nan_Afghanistan  nan_Albania  nan_Algeria  nan_Andorra
 1/22/20                0            0            0            0
 1/23/20                0            0            0            0
 1/24/20                0            0            0            0
 1/25/20                0            0            0            0
 1/26/20                0            0            0            0,
 key       nan_Afghanistan  nan_Albania  nan_Algeria  nan_Andorra
 10/13/20             1480          429         1818           57
 10/14/20             1481          434         1827           59
 10/15/20             1481          439         1827           59
 10/16/20             1485          443         1841           59
 10/17/20             1488          448         1846           59)

In [161]:
DF_countries = {c:pd.DataFrame(df_mod_t[c]) for c in df_mod_t.columns}
type(DF_countries["nan_Albania"]),DF_countries["nan_Albania"].tail()
DF2 = DF_countries.copy()
def prepare_covid(gf,column_rep):
    for k in gf.keys():
        gf[k] = gf[k].rename(columns={k:column_rep})
        gf[k]["daily_"+column_rep] = gf[k][column_rep].diff(1).fillna(0) 
        
    return pd.concat(gf.values(), keys=gf.keys())
DF3 = prepare_covid(DF2,"deaths")

In [162]:
DF3.loc[("nan_Zimbabwe","10/13/20")], DF3.index.values[-2:],DF3.apply(lambda r: r.name,axis=1)[-2:]

(deaths          230.0
 daily_deaths      0.0
 Name: (nan_Zimbabwe, 10/13/20), dtype: float64,
 array([('nan_Zimbabwe', '10/16/20'), ('nan_Zimbabwe', '10/17/20')],
       dtype=object),
 nan_Zimbabwe  10/16/20    (nan_Zimbabwe, 10/16/20)
               10/17/20    (nan_Zimbabwe, 10/17/20)
 dtype: object)

In [163]:
import time
file_name = "csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
col_name = "daily_deaths"
lags = [7,14,28,56,1]
lags2 = [7,14,28,56]

In [164]:
start = time.time()
def create_df_dict(file_name,col_name):
    df = pd.read_csv(file_name)
    df["key"] = df.apply(lambda r: str(r['Province/State'])+"_"+str(r['Country/Region']),axis=1)
    df = df.set_index("key")
    df_mod = df[df.columns[4:]]
    df_mod_t = df_mod.T
    DF_countries = {c:pd.DataFrame(df_mod_t[c]) for c in df_mod_t.columns}
    for c in DF_countries.keys():
        DF_countries[c] = DF_countries[c].rename(columns={c:col_name})
    return DF_countries
def add_lags(gf,lags,col_name):
    for c in gf.keys():
        
        for lg in lags:
            gf[c][col_name+"_lag_%d"%lg] = gf[c][col_name].shift(lg)
    return gf
round(time.time()-start,2)


0.0

In [165]:
def lag(df,col,lag,fillna_val = 0.0):
	return df[col].shift(lag).fillna(0.0)
def ma(df,col,window):
	return df[col].rolling(window).mean()
def sum(df,col,window):
    return df[col].rolling(window).sum()
def std(df,col,window):
	return df[col].rolling(window).std()
def pct_change(df,col,back):
	return df[col].pct_change(back).fillna(0.0)
def diff(df,col,back):
    return df[col].diff(back).fillna(0.0)
def local_cvt(df,col,back):
    return df[col].diff(back).fillna(0.0) -df[col].shift(back).fillna(0.0).diff(back).fillna(0.0)


In [166]:
def initialize_features_func_directional(lags,col_name):
    funcs_dict = OrderedDict()# OrderedDict()    
    for ii in lags:
        funcs_dict["%s_ma_%d"%(col_name,ii)] = {"func":ma,"params" : {"col":col_name,"window":ii}}
        funcs_dict["%s_sum_%d"%(col_name,ii)] = {"func":sum,"params" : {"col":col_name,"window":ii}}
        funcs_dict["%s_std_%d"%(col_name,ii)] = {"func":std,"params" : {"col":col_name,"window":ii}}
        #funcs_dict["%s_pct_%d"%(col_name,ii)] = {"func":pct_change,"params" : {"col":col_name,"back":ii}}
        funcs_dict["%s_diff_%d"%(col_name,ii)] = {"func":diff,"params" : {"col":col_name,"back":ii}}
        funcs_dict["%s_local_cvt_%d"%(col_name,ii)] = {"func":local_cvt,"params" : {"col":col_name,"back":ii}}
    return funcs_dict
def from_gf_to_df(frame,gf,col):
    frame[col] = frame.apply(lambda r: gf[r.name[0]].loc[r.name[1],col],axis=1)


In [151]:
from tqdm import tqdm
def genetate_directional_features(frame,gf,func_dict,lags):
    func_dict_keys = list(func_dict.keys())
    for func_name in tqdm(func_dict_keys): #func_dict.keys():
            for kk in tqdm(gf.keys()):
                 params_d = func_dict[func_name]["params"].copy()
                 params_d["df"] = gf[kk]
                 gf[kk][func_name] = func_dict[func_name]["func"](**params_d)
                 """
                 for jj in lags:
                    gf[kk][func_name+"__%d"%jj] = gf[kk][func_name].shift(jj).fillna(0)
                 """
            from_gf_to_df(frame,gf,func_name)
            """
            for lg in lags:
                from_gf_to_df(frame,gf,func_name)
                #from_gf_to_df(frame,gf,func_name+"__%d"%lg)
            """

            

In [126]:
start = time.time()
DF_all = create_df_dict(file_name=file_name,col_name=col_name)
print(round(time.time()-start,2))

1.66


In [127]:
start = time.time()
functions_dict_deaths = initialize_features_func_directional(lags,"deaths")
print(round(time.time()-start,2))

0.01


In [122]:
DF4 = DF3.copy()
start = time.time()
DF_d = create_df_dict(file_name,col_name)
print(round(time.time()-start,2))
start = time.time()
DF_d = add_lags(gf=DF_d,lags= lags,col_name = col_name)
print(round(time.time()-start,2))
start = time.time()
#print(DF4.index[-5:])
#print(DF_d['nan_Zimbabwe'].tail())#,col_name+"_lag_7"])
for lg in lags:
    from_gf_to_df(DF4,DF_d,col_name+"_lag_%d"%lg)
print(round(time.time()-start,2))

2.68
2.46
84.15


In [152]:
dict_deaths = initialize_features_func_directional(lags,"deaths")

In [153]:
start = time.time()
genetate_directional_features(DF4,DF_d,dict_deaths,lags)
print(round(time.time()-start,2))





  0%|          | 0/25 [00:00<?, ?it/s]




  0%|          | 0/267 [00:00<?, ?it/s]




 14%|█▍        | 37/267 [00:00<00:00, 366.96it/s]




 26%|██▌       | 69/267 [00:00<00:00, 350.89it/s]




 34%|███▍      | 91/267 [00:00<00:00, 285.09it/s]




 41%|████      | 110/267 [00:00<00:00, 236.57it/s]




 55%|█████▌    | 148/267 [00:00<00:00, 266.53it/s]




 68%|██████▊   | 181/267 [00:00<00:00, 280.40it/s]




 78%|███████▊  | 208/267 [00:00<00:00, 267.23it/s]




100%|██████████| 267/267 [00:00<00:00, 302.64it/s]




  4%|▍         | 1/25 [00:16<06:42, 16.77s/it]




  0%|          | 0/267 [00:00<?, ?it/s]




 16%|█▌        | 42/267 [00:00<00:00, 418.20it/s]




 25%|██▍       | 66/267 [00:00<00:00, 336.38it/s]




 37%|███▋      | 99/267 [00:00<00:00, 330.58it/s]




 48%|████▊     | 128/267 [00:00<00:00, 315.47it/s]




 60%|██████    | 161/267 [00:00<00:00, 318.69it/s]




 78%|███████▊  | 207/267 [00:00<00:00, 346.88it/s]




100%|██████████| 267/267 [00:00<00:00, 344.15it/s]

 66%|██████▌   | 176/267 [00:00<00:00, 141.83it/s]




 81%|████████  | 215/267 [00:01<00:00, 174.97it/s]




100%|██████████| 267/267 [00:01<00:00, 224.44it/s]




 48%|████▊     | 12/25 [03:25<03:36, 16.62s/it]




  0%|          | 0/267 [00:00<?, ?it/s]




 24%|██▍       | 65/267 [00:00<00:00, 645.85it/s]




 46%|████▋     | 124/267 [00:00<00:00, 626.56it/s]




 64%|██████▍   | 172/267 [00:00<00:00, 573.79it/s]




100%|██████████| 267/267 [00:00<00:00, 593.26it/s]




 52%|█████▏    | 13/25 [03:36<02:57, 14.81s/it]




  0%|          | 0/267 [00:00<?, ?it/s]




 42%|████▏     | 113/267 [00:00<00:00, 1126.84it/s]




100%|██████████| 267/267 [00:00<00:00, 902.33it/s] 




 56%|█████▌    | 14/25 [03:49<02:38, 14.42s/it]




  0%|          | 0/267 [00:00<?, ?it/s]




  9%|▉         | 24/267 [00:00<00:01, 234.67it/s]




 19%|█▊        | 50/267 [00:00<00:00, 241.35it/s]




 30%|███       | 81/267 [00:00<00:00, 257.51it/s]




 42%|████▏     | 112/267 [00:00<00:00, 270.75it/s]





100%|██████████| 267/267 [00:00<00:00, 316.14it/s]




 96%|█████████▌| 24/25 [06:14<00:14, 14.71s/it]




  0%|          | 0/267 [00:00<?, ?it/s]




  7%|▋         | 19/267 [00:00<00:01, 183.50it/s]




 13%|█▎        | 36/267 [00:00<00:01, 178.01it/s]




 23%|██▎       | 62/267 [00:00<00:01, 195.58it/s]




 32%|███▏      | 85/267 [00:00<00:00, 204.27it/s]




 42%|████▏     | 111/267 [00:00<00:00, 216.99it/s]




 49%|████▉     | 131/267 [00:00<00:00, 209.25it/s]




 58%|█████▊    | 156/267 [00:00<00:00, 217.30it/s]




 66%|██████▋   | 177/267 [00:00<00:00, 199.78it/s]




 74%|███████▍  | 197/267 [00:00<00:00, 184.04it/s]




 85%|████████▌ | 227/267 [00:01<00:00, 207.98it/s]




100%|██████████| 267/267 [00:01<00:00, 202.91it/s]




100%|██████████| 25/25 [06:33<00:00, 15.76s/it]

393.93


In [156]:
1604.85/60.
list(DF4.loc[("nan_Zimbabwe","10/13/20")])
#DF4.shape
for c in DF4.columns:
    print (c,DF4.loc[("nan_Israel","10/13/20"),c])

deaths 2055.0
deaths_lag_7 1797.0
deaths_lag_14 1528.0
deaths_lag_28 1147.0
deaths_lag_56 708.0
deaths_lag_1 2021.0
deaths_ma_7__7 1682.7142857142858
deaths_ma_7__14 1436.7142857142858
deaths_ma_7__28 1103.7142857142858
deaths_ma_7__56 673.4285714285714
deaths_ma_7__1 1901.857142857143
deaths_std_7__7 80.60751471525977
deaths_std_7__14 71.51623192374677
deaths_std_7__28 32.973293666816105
deaths_std_7__56 23.922493897704094
deaths_std_7__1 82.23427625069314
deaths_pct_7__7 0.17604712041884807
deaths_pct_7__14 0.18910505836575875
deaths_pct_7__28 0.10288461538461546
deaths_pct_7__56 0.13826366559485526
deaths_pct_7__1 0.15025611838360842
deaths_diff_7__7 269.0
deaths_diff_7__14 243.0
deaths_diff_7__28 107.0
deaths_diff_7__56 86.0
deaths_diff_7__1 264.0
deaths_7__7 26.0
deaths_7__14 105.0
deaths_7__28 24.0
deaths_7__56 25.0
deaths_7__1 14.0
deaths_ma_14__7 1559.7142857142858
deaths_ma_14__14 1330.5
deaths_ma_14__28 1054.642857142857
deaths_ma_14__56 633.1428571428571
deaths_ma_14__1 1773